### 爬取校招2021信息 by 喝醉的龙猫

#### 导入库

In [1]:
import pandas as pd #这是本次将数据保存为表格形式，进行数据清洗，与录入excel文件所需要的库
import requests #让python请求网页的库
from bs4 import BeautifulSoup #解析html代码的库
import re #正则表达式的库，爬取校招信息对应链接时用到
import time #生成日期，保存的时候用

#### 数据抓取

In [2]:
namelist=[]
datelist=[]
link=[]
# range: 第一页到第二十三页，页数范围可以自己改，信息由新到旧每页100个。
for i in range(1,24):

    url = 'http://www.yingjiesheng.com/2021/index-'+str(i)+'.html'
    #print('{}'.format(i))
    page = requests.get(url).content
    soup = BeautifulSoup(page,'html')
    #print('good to go')
    company = soup.find_all('div', class_='s_clear tit floatl')
    for i in company:
        namelist.append(i.text)

    for i in range(len(company)):
        link.append(company[i].find_all('a')[0]['href'])

    date = soup.find_all('div', class_='box1 bg1')
    for i in date:
        datelist.append(i.text)
    #print('done')
print('done')

done


#### 数据清洗

In [3]:
df = pd.DataFrame([namelist,datelist,link]).T #先把数据储存进表格里
df.columns=['name','date','link']
df['date'] = df.date.apply(lambda x: x.strip())
df.drop([0,1],axis=0,inplace=True) #去掉前两行广告

linklist = df['link'].tolist() #修改链接让它可以直接访问

for i in range(len(linklist)):
    if re.search('^/job-',linklist[i]):
        linklist[i] = ('http://www.yingjiesheng.com'+linklist[i])
    else:
        pass

df['link'] = linklist 

df['date'] = df.date.apply(lambda x: x.replace('网申','')) #去除首尾的换行符和‘网申’二字
df

,name,date,link
2,【北京上海成都深圳】腾讯2021游戏美术校园招聘,2020-07-07启动,http://www.yingjiesheng.com/job-004-680-208.html
3,【北京上海】美团2021校园招聘-北斗计划,2020-07-07启动,http://www.yingjiesheng.com/job-004-680-206.html
4,【江西】兴业银行南昌分行2020校园招聘,2020-07-07_07-17,http://www.yingjiesheng.com/job-004-680-202.html
5,【黑龙江】招商银行哈尔滨分行2020校园招聘柜员,2020-07-07_07-17,http://www.yingjiesheng.com/job-004-680-199.html
6,【青海】招商银行西宁分行2020实习生招聘,2020-07-06_07-12,http://www.yingjiesheng.com/job-004-678-838.html
...,...,...,...
2295,【全国海外】小米2020校园招聘,2019.7.25启动,http://www.yingjiesheng.com/job-004-243-942.html
2296,【全国】腾讯2020校园招聘,2019.7.25-9.15,http://www.yingjiesheng.com/job-004-243-915.html
2297,【北京上海广州深圳】腾讯2020校园招聘产品经理培训生,2019.7.25-10.11,http://www.yingjiesheng.com/job-004-243-914.html
2298,【北京上海香港】Goldman Sachs高盛2020校园招聘,2019.7.24-10.13,http://www.yingjiesheng.com/job-004-243-275.html


#### 存储Excel文件到本地

In [4]:
timestamp = time.strftime('%m%d',time.localtime())
filename = 'recruit'+timestamp+'.xlsx'
df.to_excel(filename,index=False,encoding='utf-8-sig')